# TFM
Graph Convolutional Networks for Human Identification Based on Multichannel ECG Signals
--------------------------
José Ignacio Díez Ruiz (100487766)

## Set up

### Colab set-up

#### Mounting colab

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

#### Installing pytorch geometric

In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

%pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-geometric
%pip install torch-geometric-temporal

### Generic set-up

#### Loading required packages

In [ ]:
import os
import numpy as np
from scipy import signal
from tqdm import tqdm
from sklearn.metrics import mutual_info_score
from scipy.io import loadmat
from keras.utils import pad_sequences
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import torch
from torch import nn
from torch.nn import Conv1d, MaxPool1d, Linear, Dropout
import torch.nn.functional as F

import torch_geometric as tg
from torch_geometric.nn import GCNConv, global_mean_pool, global_sort_pool
from torch_geometric.utils import remove_self_loops
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from torch_geometric_temporal.nn.recurrent import GConvGRU
# ----- seting the device and seeds -----------------------------------

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
np.random.seed(33)
torch.manual_seed(42)

#### Helper functions definition

In [ ]:
def load_mat(filename):
    """Loads a .mat file as a numpy array"""
    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)
    return data

def import_ecg_data(directory, seconds_per_window=5, default_window_size=128, n_files_to_load=None):
    """
    Imports all .mat file containing ECG data of a given directory as a list.
    It also returns the names of the loaded files, as they may be useful to
    use as labels.

    When the next window is smaller than thw given window size, the
    function skips to the next sample.

    Parameters
    ----------
    directory: string or Path
        Path where .mat files are stored
    seconds_per_window: int
        Number of seconds to take for every window
    default_window_size: int
        Number of samples for every window.
        Only used if no .hea file in found in the directory
        with metadata.
    n_files_to_load: int
        Number of files to load in the directory.
        If no number is specificied, all files will be loaded.
        If a number is specified, the load of the files
        will be in alphabetic order.
    trunc:
    pad:
    """
    print("Starting ECG import...")
    ecgs = []
    filenames = []
    files_loaded = 0
    for ecgfilename in tqdm(sorted(os.listdir(directory))):
        filepath = directory + os.sep + ecgfilename
        if filepath.endswith(".mat"):
            try:
                hea_filepath = directory + os.sep + Path(filepath).stem + ".hea"
                with open(hea_filepath, "r") as f:
                    sampling_frequency = int(f.readline().split()[2]) # number of samples per second
                    window_size = sampling_frequency * seconds_per_window
            except Exception as e:
                window_size = default_window_size * seconds_per_window
            data = load_mat(filepath)
            current_index = 0
            previous_index = 0
            while current_index <= (data[0].shape[0] - window_size):
                current_index += window_size
                window = data[:,previous_index:current_index]
                window = pad_sequences(window, truncating="post", padding="post")
                ecgs.append(window)
                filenames.append(ecgfilename.split(".")[0])
                previous_index = current_index
            files_loaded += 1
            if n_files_to_load is not None and files_loaded == n_files_to_load:
                break
    print("Finished!")
    return ecgs, filenames

def calc_MI_matrix(data, bins=200):
    """
    It computes the Mutual Information matrix for 2 variables

    Parameters
    ----------
    data: numpy ndarray with 2 dimensions
        Where the data of the variables is stored
    bins: int
        Number of bins of the histogram used to estimate the mutual information score.
        Default: 200
    """
    dims = data.shape[0]
    mi_matrix = np.zeros((dims,dims))

    for i in range(dims):
        for j in range(dims):
            x = np.array(data[i])
            y = np.array(data[j])
            c_xy = np.histogram2d(x, y, bins)[0]

            mi_matrix[i,j] = mutual_info_score(None, None, contingency=c_xy)

    return mi_matrix

def train(train_loader, edge_weights=True):
    """
    It trains a pytorch geometric model.

    """
    model.train()

    for data in train_loader:
        data = data.to(device)
        if edge_weights:
            outputs = model(data.x, data.edge_index, data.edge_weight, data.batch)
        else:
            outputs = model(data.x, data.edge_index, data.batch)
        loss = criterion(outputs, data.y.long())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def test(loader, edge_weights=True):
    """
    It tests a pytorch_geometric model for a given loader.

    """
    model.eval()

    total_correct  = 0
    total_samples  = 0
    true_positive  = [0] * num_classes
    false_positive = [0] * num_classes
    false_negative = [0] * num_classes
    CM = 0 # define an empty confussion matrix

    for data in loader:  # Iterate in batches over the training/test dataset.
        data = data.to(device)
        if edge_weights:
            outputs = model(data.x, data.edge_index, data.edge_weight, data.batch)
        else:
            outputs = model(data.x, data.edge_index, data.batch)
        preds = outputs.argmax(dim=1)  # Use the class with highest probability.
        total_correct += int((preds == data.y).sum())  # Check against ground-truth labels.
        total_samples += len(data.y)
        CM += confusion_matrix(data.y.cpu(), preds.cpu(), labels=np.unique(labels))
        for i in range(num_classes):
            true_positive[i]  += ((preds == i) & (data.y == i)).sum().item()
            false_positive[i] += ((preds == i) & (data.y != i)).sum().item()
            false_negative[i] += ((preds != i) & (data.y == i)).sum().item()

    accuracy    = total_correct / total_samples
    precision   = [tp / (tp + fp + 1e-6) for tp, fp in zip(true_positive, false_positive)]
    recall      = [tp / (tp + fn + 1e-6) for tp, fn in zip(true_positive, false_negative)]
    f1_scores   = [2 * (pr * re) / (pr + re + 1e-6) for pr, re in zip(precision, recall)]

    return accuracy, precision, recall, f1_scores, CM

def pyg_ei_fc(num_nodes, self_loops=True):
    """
    Edge index matrix generator for a fully connected graph in pytorch geometric format

    Parameters
    ----------
    num_nodes: int
        Number of nodes of the graph.
    self_lopps: bool
        If the graph has self loops or not.
    """
    
    # Generate all possible combinations of edges (complete graph)
    if self_loops:
        edges = [(i, j) for i in range(num_nodes) for j in range(num_nodes)]
    else:
        edges = [(i, j) for i in range(num_nodes) for j in range(num_nodes) if i != j]

    # Separate the source and target nodes into two separate lists
    src, tgt = zip(*edges)

    # Convert the lists to PyTorch tensors
    src = torch.tensor(src, dtype=torch.long)
    tgt = torch.tensor(tgt, dtype=torch.long)

    # Concatenate the source and target tensors to create the edge index
    edge_index = torch.stack([src, tgt], dim=0)

    return edge_index

def indices_generator(first, last, proportion):
    """
    Generates non-overlaping train and test indices in range [first, last] (both inclusive)
    """
    n = round(num_ecg_signals * proportion)

    # n random indices in the range
    train_indices = np.random.randint(first, last, n)
    train_indices = np.random.choice(np.arange(first, last), size=n, replace=False)

    # obtain the complementary of the set
    all_indices = np.arange(first, last)
    test_indices = np.setdiff1d(all_indices, train_indices)

    return train_indices, test_indices

## Some GNN Architectures Examples

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(33)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes) # dataset.num_classes

    def forward(self, x, edge_index, edge_weight, batch):
        x = self.conv1(x, edge_index, edge_weight)
        x = x.relu()
        x = self.conv2(x, edge_index, edge_weight)

        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        x = F.dropout(x, p=0.3, training=self.training)
        x = self.lin(x)

        return x

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(33)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes) # dataset.num_classes

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        x = F.dropout(x, p=0.3, training=self.training)
        x = self.lin(x)

        return x

In [ ]:
class RecurrentGCN(torch.nn.Module):

    def __init__(self, node_features, num_classes):
        super(RecurrentGCN, self).__init__()
        self.recurrent_1 = GConvGRU(node_features, 256, 5)
        self.linear1 = torch.nn.Linear(256, 256)
        self.linear_final = torch.nn.Linear(256, num_classes)

    def forward(self, x, edge_index, edge_weight, batch):
        x = self.recurrent_1(x, edge_index, edge_weight)
        x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        x = global_mean_pool(x, batch)

        x = F.dropout(x, training=self.training)
        x = self.linear1(x)
        x = self.linear_final(x)
        # return F.log_softmax(x, dim=1)
        return x

## Data loading and model training


### Data loading

In [ ]:
database_path = "/path/to/database"

N_FILES_TO_LOAD = 32 
SECONDS_PER_WINDOW = 5

database_data, database_filenames = import_ecg_data(database_path,
                                                    seconds_per_window=SECONDS_PER_WINDOW,
                                                    n_files_to_load=N_FILES_TO_LOAD)

database_data = np.asarray(database_data)
database_data = torch.from_numpy(database_data) # convert the data to a PyTorch tensor
num_ecg_signals, num_ecg_leads, num_samples = database_data.shape

# The calculation of the MI matrix is ommited and the matrix is loaded from an external file

# mi = [calc_MI_matrix(database_data[i]) for i in range(num_ecg_signals)]

# with open('mi_matrices.npy', 'wb') as f:
#     np.save(f, np.array(mi))

# with open('mi_matrices.npy', 'rb') as f:
#      mi = np.load(f)

samples_per_file = 1800/SECONDS_PER_WINDOW

# Select only the needed individuals will be selected from the MI matrix

mi = mi[0:int(samples_per_file*N_FILES_TO_LOAD)]

# We need to reshape to (num_ecg_signals, num_ecg_leads * num_samples)

database_data = database_data.view(num_ecg_signals, -1)
num_classes = len(np.unique(database_filenames, return_counts=True)[0])

print(f"num_ecg_signals: {num_ecg_signals}\nnum_classes: {num_classes}")

In [ ]:
# ------ WE NEED TO GENERATE THE EDGE_INDEX IN ORDER TO CREATE THE GRAPH DATASET ------

num_nodes = num_ecg_leads
edge_index = pyg_ei_fc(num_nodes=num_ecg_leads, self_loops=True)

# ------ NEXT, LETS CREATE THE DATASET ---------------------------------------------

graph_list = [] # empty list to store the graphs

# Generating the tensor of labels

real_labels = [int(label.split("I")[1]) for label in database_filenames] # the labels should between 0 and N-1
labels = torch.tensor(np.repeat(range(num_classes), np.unique(real_labels, return_counts=True)[1]))


for signal_idx in range(num_ecg_signals):
    feature_matrix = database_data[signal_idx].reshape(num_ecg_leads, num_samples)  # Reshape to (num_ecg_leads, num_samples)

    # Convert feature_matrix and adjacency matrix to PyTorch Geometric data format
    x = feature_matrix.clone().detach().float()
    edge_index  = edge_index  # fully connected graph with self loops
    edge_weight = torch.from_numpy(mi[signal_idx].flatten()).float() # importante el float
    # https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-double-but-got-scalar-type-float-for-argument-2-weight/38961/13

    y = labels[signal_idx].clone().detach()

    # Create a PyTorch Geometric data object
    graph = tg.data.Data(x=x, edge_index=edge_index, edge_weight=edge_weight, y=y)
    graph_list.append(graph)

dataset = tg.data.Batch.from_data_list(graph_list)

In [ ]:
# We need to create both train and test dataset.
# Recall that the train dataset will be used for cross validation, hence for optimizing the parameters of the model.

train_indices, test_indices = indices_generator(0, num_ecg_signals, 0.9)

train_dataset_complete = dataset[list(train_indices)]
test_dataset = dataset[list(test_indices)]

### Training and validating the models

In [ ]:
# ---------- DEFINITION OF THE HYPERPARAMETERS ------------------
learning_rate = 0.02
weight_decay  = 0
batch_size    = 128 # Batch size for the loaders
num_epochs    = 50
num_folds     = 5  # Number of folds for cross-validation
criterion     = nn.CrossEntropyLoss()
# ---------- LISTS TO STORE THE RESULTS ------------------------------
training_accuracy_list      = []
training_precision_list     = []
training_recall_list        = []
training_f1_list            = []
validation_accuracy_list    = []
validation_precision_list   = []
validation_recall_list      = []
validation_f1_list          = []
# --------- CREATE THE TEST LOADER FOR FUTURE TESTING -----------------------------------
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

kf = KFold(n_splits=num_folds, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kf.split(dataset)):

    train_dataset = dataset[list(train_indices)]
    val_dataset   = dataset[list(val_indices)]

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    model = GCN(hidden_channels=512)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        train(train_loader, edge_weights=True)

        with torch.no_grad():
            accuracy_train, precision_train, recall_train, f1score_train, _ = test(train_loader, edge_weights=True)
            accuracy_val, precision_val, recall_val, f1score_val, _         = test(val_loader, edge_weights=True)

        print(f"""
        Fold [{fold+1}/{num_folds}] - Epoch [{epoch+1}/{num_epochs}]\n
        Train Accuracy: {accuracy_train:.3f} - Validation Accuracy: {accuracy_val:.3f}\n
        Validation Precision: {np.mean(precision_val):.3f} - Validation Recall: {np.mean(recall_val):.3f} - Validation F1: {np.mean(f1score_val):.3f}\n
        -----------------------------------------
        """)

        # If it is inside the last epoch, append the results to a list to later visualization
        if epoch == (num_epochs - 1):
            training_accuracy_list.append(accuracy_train)
            training_precision_list.append(np.mean(precision_train))
            training_recall_list.append(np.mean(recall_train))
            training_f1_list.append(np.mean(f1score_train))

            validation_accuracy_list.append(accuracy_val)
            validation_precision_list.append(np.mean(precision_val))
            validation_recall_list.append(np.mean(recall_val))
            validation_f1_list.append(np.mean(f1score_val))

In [ ]:
# Plot a confusion matrix

cm_plot = ConfusionMatrixDisplay(CM, display_labels=[])
cm_plot.plot(cmap=plt.cm.Blues, include_values=False)

plt.title('Confusion Matrix')
plt.show()